In [1]:
import numpy as np
from numba import njit
from random import randint
from time import time
from re import search

In [31]:
@njit()
def run():
    d = np.zeros((64,64), dtype=np.uint8) #only 1 or 0
    u = np.zeros((64,64), dtype=np.int8) #can be negative
    v = np.zeros((64,64), dtype=np.uint8) #only 1 or 0
    s = np.zeros(64,dtype=np.uint8)
    m = np.int8(-4)

    for i in range(64): #initialize
        for j in range(64):
            x = (i//8)-(j//8)
            y = (i%8)-(j%8)
            if(((x*x)+(y*y))==5):
                d[i][j] = 1
                d[j][i] = 1
                v[i][j] = randint(0,1)
                v[j][i] = v[i][j]
    
    for counter in range(100000):
        for i in range(64):
            s[i] = np.sum(v[i]) #option 2: do the for loop

        for i in range(64):
            for j in range(64):
                if d[i][j]:
                    u[i][j] += (4-s[i]-s[j])
                    v[i][j] = (u[i][j]>3)+(u[i][j]&m==0)*(v[i][j]) 
                    # ^ fastest way I think
        
        if counter % 2000 == 0:
            if np.max(np.absolute(u)) > 100: 
                print("diverge")
                return [(np.int64(x), np.int64(x)) for x in range(0)]
            b = 0
            for i in range(64):
                for j in range(64):
                    b += np.absolute(d[i][j]*(4-(s[i]+s[j])))     
            if b == 0:
                soln = [(np.int64(x), np.int64(x)) for x in range(0)]
                for i in range(64):
                    for j in range(i+1,64):
                        if v[i][j]:
                            soln.append((i,j))
                return soln
    return [(np.int64(x), np.int64(x)) for x in range(0)]

In [32]:
def get_sol():
    start = time()
    runs = 0;
    while True:
        soln = run() #list of edges
        runs += 1;
        if len(soln) > 0:
            sol = [1] + [0]*63
            cur=0
            for q in range(1,64):
                for edge in soln:
                    if cur in edge:
                        oth = edge[0] if edge[1] == cur else edge[1]
                    if sol[oth]==0:
                        sol[oth]=q+1;
                        cur=oth
                        break
#             if 0 not in sol:
            return "".join(["{:>2} ".format(sol[i]) + ("\n" if (i%8==7) else "") for i in range(64)]) + "{:.2f} seconds {} runs\n".format(time()-start, runs)

In [33]:
def big_run(N):
    for i in range(N):
        print("{} out of {} finished".format(i,N), end="\r");
        f = open("fastnn.txt","a")
        f.write(get_sol())
        f.close()
    analyze()

In [34]:
def analyze():
    f = open("fastnn.txt","r")
    s = f.read().split("\n")
    f.close()
    npt = np.array(sorted([float(time_taken.split(' ')[0]) for time_taken in s if search("[0-9.]+ seconds",time_taken) is not None]))
    print("len: {}\navg: {}\nstd: {}\nmed: {}\n85th: {}".format(len(npt),np.average(npt), np.std(npt), np.median(npt), np.percentile(npt,85)))

In [37]:
print(get_sol())

 1  0 15 10  3  8 25 22 
14 11  2  0 26 23  4  7 
 0 60 13 16  9  6 21 24 
12 17  0 27 56 19 44  5 
59 32 55 18 45 42 49 20 
54 35 30 57 28 39 46 43 
31 58 33 52 37 48 41 50 
34 53 36 29 40 51 38 47 
5.16 seconds 7 runs



In [12]:
analyze()

len: 1000
avg: 18.37755
std: 18.467005374383252
med: 13.0
85th: 35.0
